In [155]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [156]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [157]:
def cspan(IDTEXT): 
    span = 6
    if IDTEXT[0] == '0':
        return '6', '' 
    IDTEXT=IDTEXT.split('.')
    if len(IDTEXT) == 1:
        return '3','' 
    if len(IDTEXT) == 2:
        return '2',f'<td colspan="1">&nbsp</td>'   
    if len(IDTEXT) > 2:
        return '1',f'<td colspan="2">&nbsp</td>' 
    return str(span), ele
cspan('1.2.3')   

('1', '<td colspan="2">&nbsp</td>')

In [158]:
query = """ \
	SELECT DISTINCT  
	Q_TypeCode AS FTPE,  IDTEXT,  SortPos, 
	PK_Question [{0}], 
	REPLACE(IDTEXT, '.','_') AS [{1}], 
	CONVERT(NVARCHAR(5), PK_PickListType) + '' AS [{2}],
    '3' AS [{3}],
	CONVERT(NVARCHAR(5), PK_Question) + ' ' + IDTEXT + ' ' +  LEFT(QTEXT, 50) AS [{4}],
	' r_idt_' + REPLACE(IDTEXT, '.','_') +  ' ct_' +  Q_TypeCode  AS [{5}]
	FROM  vwQuestions WHERE PK_FORM LIKE '2022-A-HVA'   ORDER BY SortPos ASC
"""  
df = get_df(query) 
df.fillna("",inplace=True)
df.replace(to_replace=[None], value=np.nan, inplace=True) 
df = df.astype(str)
df.to_csv(r'C:\Users\Tim\Documents\1802\HVA\form1a.csv', index=False)  
df[0:4]

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,LABEL,0.1,1,28200,0_1,,3,28200 0.1 HVA Information,r_idt_0_1 ct_LABEL
1,TXT,1,2,28201,1,,3,28201 1 What is the purpose of the HVA?,r_idt_1 ct_TXT
2,MULTICHECKBOX,1.1,3,28202,1_1,356,3,28202 1.1 Please Select the Applicable Purpose...,r_idt_1_1 ct_MULTICHECKBOX
3,TXT,1.1.1,4,28203,1_1_1,,3,28203 1.1.1 If .GOV Domain Registration System...,r_idt_1_1_1 ct_TXT


In [159]:
df = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\form1a.csv', dtype=str)  
df.fillna("",inplace=True)
df = df.astype(str) 
df[0:4]

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,LABEL,0.1,1,28200,0_1,,3,28200 0.1 HVA Information,r_idt_0_1 ct_LABEL
1,TXT,1,2,28201,1,,3,28201 1 What is the purpose of the HVA?,r_idt_1 ct_TXT
2,MULTICHECKBOX,1.1,3,28202,1_1,356,3,28202 1.1 Please Select the Applicable Purpose...,r_idt_1_1 ct_MULTICHECKBOX
3,TXT,1.1.1,4,28203,1_1_1,,3,28203 1.1.1 If .GOV Domain Registration System...,r_idt_1_1_1 ct_TXT


In [160]:

st=''
for i,r in df.iterrows():
    tmp = f"C:\_som\_src\{r['FTPE']}.aspx"
    r0 = r['{0}']
    with open(tmp, 'r') as f:
        tmp = f.read()
    csp = cspan(r["IDTEXT"])
    tmp = re.sub('colspan="\d"','colspan="{3}"',tmp)
    tmp = tmp.replace("<!--{cols}-->", str(csp[1])) 
    tmp = tmp.replace("{3}", str(csp[0])) 
    tmp = tmp.replace("{0}", str(r["{0}"])) 
    tmp = tmp.replace("{1}", str(r["{1}"])) 
    tmp = tmp.replace("{2}", str(r["{2}"]))  
    tmp = tmp.replace("{4}", str(r["{4}"]))  
    tmp = tmp.replace("{5}", str(r["{5}"]))  
    st=st+tmp
with open(r'C:\Users\Tim\Documents\1802\HVA\page.aspx', 'w') as f:
    f.write(st) 

In [161]:
f1A=''
with open(fr'C:\_som\_src\1A.aspx', mode="r", encoding="utf-8") as f:
    f1A = f.read()  
f1A = f1A.replace("<!--BODY-->", st)
with open(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\HVA\2022\2022_A_HVA_1A.aspx', 'w') as f:
    f.write(f1A) 
print( f1A )

<%@ Page Title="" Language="vb" AutoEventWireup="false" MasterPageFile="~/Form.Master" CodeBehind="2022_A_HVA_1A.aspx.vb" Inherits="CyberScope._2022_A_HVA_1A" %>
<%@ Register Src="../../CustomControls/CBlabel.ascx" TagName="CBlabel" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBoptionalLinks.ascx" TagName="CBoptionalLinks" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBButtPanel2.ascx" TagName="CBButtPanel" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBtext2.ascx" TagName="CBtext2" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBPickList2.ascx" TagName="CBPickList2" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CbPickListMultiSelect.ascx" TagName="CbPickListMultiSelect" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBYesNo.ascx" TagName="CBYesNo" TagPrefix="uc" %>
<asp:Content ID="Content1" ContentPlaceHolderID="ContentPlaceHolder1" Runat="Server">
    <asp:Panel ID="Panel1" runat="server">
             
        <ta